In [1]:
!pip install -U transformers

In [2]:
pip install transformers==4.36.2 accelerate torch

  Using cached transformers-4.36.2-py3-none-any.whl.metadata (126 kB)
  Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.36.2-py3-none-any.whl (8.2 MB)
Using cached tokenizers-0.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.0
    Uninstalling transformers-4.55.0:
      Successfully uninstalled transformers-4.55.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.0.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.


In [3]:
pip install sentencepiece

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Open, chat-tuned TinyLlama[1]

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [5]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_no

In [6]:
# Helper: generate with chat messages using TinyLlama's chat template
def chat_generate(messages, max_new_tokens=256, temperature=0.7, top_p=0.95, top_k=50, repetition_penalty=1.1):
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)  # per TinyLlama docs[1][6]
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        repetition_penalty=repetition_penalty,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [17]:
def generate_poem(prompt_text="Write me a poem about Machine Learning."):
    messages = [
        {"role": "system", "content": "You are a helpful, creative assistant."},
        {"role": "user", "content": prompt_text}
    ]
    return chat_generate(messages, max_new_tokens=200)

print(generate_poem("Write me a poem about Machine Learning."))

<|system|>
You are a helpful, creative assistant. 
<|user|>
Write me a poem about Machine Learning. 
<|assistant|>
Machine Learning is the art and science of designing algorithms to learn from data without being programmed explicitly. It’s a fascinating field that has revolutionized many industries, including finance, healthcare, and automotive. Here's a poem inspired by the wonders of machine learning:

Machine learning rules with precision,
A powerful tool that can be trusted.
It learns on its own, never stops,
Its innovations transforming life around.

From fraud detection in financial institutions to cancer diagnosis,
Machine learning delivers results beyond compare.
It helps us understand our customers better,
And make informed decisions with ease.

The possibilities are endless,
As we tap into this powerful tool.
But let's not forget to protect it,
Lest it gets overwhelmed with data.

So let's always keep a watchful eye,
On the data


In [8]:
# 2) AI-content detection: Instructional prompt (first version)
instruction_prompt_v1 = (
    "You are a content detection expert.\n"
    "Your task is to determine whether the following student-written paragraph is generated by an AI or a human.\n"
    "Be honest, unbiased, and give a reason for your decision.\n\n"
    "Student Paragraph:\n"
    "\"\"\"\n"
    "{}\n"
    "\"\"\"\n\n"
    "Your Analysis:"
)

In [9]:
def detect_ai_content_instructional_v1(student_paragraph):
    user_text = instruction_prompt_v1.format(student_paragraph)
    messages = [
        {"role": "system", "content": "You are a rigorous content detection expert."},
        {"role": "user", "content": user_text}
    ]
    return chat_generate(messages, max_new_tokens=250)

In [10]:
student_input_1 = "Machine learning allows systems to learn from data and make predictions without being explicitly programmed."
print(detect_ai_content_instructional_v1(student_input_1))

<|system|>
You are a rigorous content detection expert. 
<|user|>
You are a content detection expert.
Your task is to determine whether the following student-written paragraph is generated by an AI or a human.
Be honest, unbiased, and give a reason for your decision.

Student Paragraph:
"""
Machine learning allows systems to learn from data and make predictions without being explicitly programmed.
"""

Your Analysis: 
<|assistant|>
Based on the given material, I believe that the student's statement "Machine learning allows systems to learn from data and make predictions without being explicitly programmed" is likely generated by an AI. The use of the phrase "without being explicitly programmed" suggests that the machine learning system has been programmed with a set of rules or algorithms. This kind of language is commonly used in AI programs to indicate the lack of explicit instructions or program logic. Additionally, the sentence "machine learning allows systems to learn from data" i

In [11]:
student_input_1 = "Machine learning enables computers to learn from data and improve performance without explicit programming."
print(detect_ai_content_instructional_v1(student_input_1))

<|system|>
You are a rigorous content detection expert. 
<|user|>
You are a content detection expert.
Your task is to determine whether the following student-written paragraph is generated by an AI or a human.
Be honest, unbiased, and give a reason for your decision.

Student Paragraph:
"""
Machine learning enables computers to learn from data and improve performance without explicit programming.
"""

Your Analysis: 
<|assistant|>
Based on the given material, it can be concluded that the student's paragraph is likely written by an AI. The language used in the sentence "Machine learning enables computers to learn from data and improve performance without explicit programming" strongly suggests that the AI is responsible for generating this text. This is because machine learning involves the application of algorithms to data in order to identify patterns and make predictions. The sentence also uses strong noun phrases such as "machine learning" and "data" that reinforce the idea that an 

In [12]:
student_input_1 = "Machine learning is being used by many fields around the globe, such as in major field like medical."
print(detect_ai_content_instructional_v1(student_input_1))

<|system|>
You are a rigorous content detection expert. 
<|user|>
You are a content detection expert.
Your task is to determine whether the following student-written paragraph is generated by an AI or a human.
Be honest, unbiased, and give a reason for your decision.

Student Paragraph:
"""
Machine learning is being used by many fields around the globe, such as in major field like medical.
"""

Your Analysis: 
<|assistant|>
Based on the provided student-generated paragraph, I believe it to be generated by an AI since there is no indication of any human input beyond the use of machine learning. The use of "machine learning" in the sentence is straightforward and does not suggest any other human involvement in generating this text. Additionally, the sentence structure and tone indicate that it was generated by an AI rather than a human. Therefore, based on these factors, I would conclude that the given passage is likely generated by an AI.


In [14]:
# 3) AI-content detection: Structured, step-by-step rubric (second version)
instruction_prompt_v2 = (
    "You are a content detection expert specialized in identifying AI-generated writing.\n"
    "Your task is to determine whether the following student-written paragraph is generated by an AI or a human.\n\n"
    "Follow these guidelines:\n"
    "1. Be honest, unbiased, and support your decision with reasoning.\n"
    "2. Analyze the paragraph based on the following dimensions:\n"
    "   - Writing Style: Assess natural flow, consistency, and tone.\n"
    "   - Language Complexity: Look for vocabulary depth, sentence variation, and abstraction.\n"
    "   - Logical Flow: Examine coherence, transitions, and clarity.\n"
    "   - Originality: Is the content specific and insightful or vague and generic?\n"
    "   - Emotional Tone: Consider empathy, nuance, or personal perspective.\n"
    "   - Red Flags: Repetition, unnatural phrasing, overuse of buzzwords, lack of specifics.\n"
    "3. Use critical thinking and forensic linguistic skills.\n"
    "4. Clearly conclude: Was it written by a Human or AI?\n\n"
    "Student Paragraph:\n"
    "\"\"\"\n"
    "{}\n"
    "\"\"\"\n\n"
    "Step-by-Step Analysis:\n"
    "- Writing Style:\n"
    "- Language Complexity:\n"
    "- Logical Flow:\n"
    "- Originality:\n"
    "- Emotional Tone:\n"
    "- Red Flags:\n\n"
    "Final Verdict (Human or AI) and Explanation:"
)

In [15]:
def detect_ai_content_instructional_v2(student_paragraph):
    user_text = instruction_prompt_v2.format(student_paragraph)
    messages = [
        {"role": "system", "content": "You are a rigorous content detection expert who follows instructions precisely."},
        {"role": "user", "content": user_text}
    ]
    return chat_generate(messages, max_new_tokens=300)

In [16]:
student_input_2 = (
    "I still remember the first time I used an AI tool, it was amazing, but also a little weird. "
    "It felt nice to have help from a machine, but it made me think about how we think. "
    "Maybe the real question is not what AI can do, but what we might stop doing ourselves."
)
print(detect_ai_content_instructional_v2(student_input_2))

<|system|>
You are a rigorous content detection expert who follows instructions precisely. 
<|user|>
You are a content detection expert specialized in identifying AI-generated writing.
Your task is to determine whether the following student-written paragraph is generated by an AI or a human.

Follow these guidelines:
1. Be honest, unbiased, and support your decision with reasoning.
2. Analyze the paragraph based on the following dimensions:
   - Writing Style: Assess natural flow, consistency, and tone.
   - Language Complexity: Look for vocabulary depth, sentence variation, and abstraction.
   - Logical Flow: Examine coherence, transitions, and clarity.
   - Originality: Is the content specific and insightful or vague and generic?
   - Emotional Tone: Consider empathy, nuance, or personal perspective.
   - Red Flags: Repetition, unnatural phrasing, overuse of buzzwords, lack of specifics.
3. Use critical thinking and forensic linguistic skills.
4. Clearly conclude: Was it written by a